In [1]:
import cv2
import matplotlib.pyplot as plt
from skimage import exposure
import os
import numpy as np

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
csv_file = 'dicom_files/classified_view.csv'
png_folder = 'pngs'

df = pd.read_csv(csv_file)
df = df[df['view'] == 'front']

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True, resize_x = 1024, resize_y = 1024):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    img = data - np.min(data)
    
    resized = cv2.resize(img, (resize_x, resize_y), interpolation=cv2.INTER_AREA)
        
    return resized

In [ ]:
#df = df.head(3)
for i, row in df.iterrows():
    dicom_file = row['Filename'][3:]
    
    png_file = os.path.join(png_folder, str(i).zfill(6))
    
    img_pixel_array = read_xray(dicom_file)

    f, axarr = plt.subplots(1,1)
        
    # normal
    print(dicom_file)
    axarr.imshow(img_pixel_array, cmap='gray')
    axarr.axis('off')
    plt.imsave(str(png_file) + '.png', img_pixel_array,  cmap='gray')
    
    # clahe
    #img_pixel_array = exposure.equalize_adapthist(img_pixel_array/np.max(img_pixel_array))
    #axarr[1].imshow(img_pixel_array, cmap='gray')
    #axarr[1].axis('off')
    #plt.imsave(str(png_file) + '_clahe.png', img_pixel_array,  cmap='gray')
        
    # histogram normalization
    #img_pixel_array = exposure.equalize_hist(img_pixel_array)
    #axarr[2].imshow(img_pixel_array, cmap='gray')
    #axarr[2].axis('off')
    #plt.imsave(str(png_file) + '_hist_norm.png', img_pixel_array,  cmap='gray')
    
    plt.show()    
    

In [5]:
df['png'] = df.index
df['path_png'] = df.png.apply(lambda x: os.path.join(png_folder, str(x).zfill(6)) + '.png')
df = df.drop(columns=['png'])

In [6]:
df.to_csv('exams_pngs.csv', index=False)